In [26]:
import pandas as pd

orig_file = pd.read_csv('ref75p.txt', delimiter=' ', header = None)
orig_file.to_csv ('75p-Ref.csv', index=None)

inf_file = pd.read_csv ('imputed_vals_110620-50p.txt', delimiter=' ', header = None)
inf_file.to_csv ('75p-InfCSV.csv', index=None)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 2, saw 2


In [ ]:
'''print(type(orig_file))

#prints first SNP in orig_file
print(orig_file.loc[0])
print('here')
#first index is SNP, second index is number into the SNP (but we start with 5 header type things)
print(orig_file.loc[0,5])
print("New line")
print(orig_file.loc[0:2,5:6])'''

In [ ]:
#we want to calculate the probability distributions of the orig and inferred genotypes
#inferred data gives three nums, while orig data gives two nums
#naive solution

pr_maj = []
pr_hetero = []
pr_min = []

for loc in range(5000):
    f_maj = 0
    f_hetero = 0
    f_min = 0
    first_SNP = orig_file.loc[loc, 5:]

    for i in range(int((len(first_SNP))/2)):
        if (first_SNP.loc[5 +2*i] + first_SNP.loc[5+2*i+1]) == 0:
            f_maj += 1
        elif (first_SNP.loc[5 +2*i] + first_SNP.loc[5+2*i+1]) == 1:
            f_hetero += 1
        elif (first_SNP.loc[5 +2*i] + first_SNP.loc[5+2*i+1]) == 2:
            f_min += 1
    pr_maj.append(f_maj / len(first_SNP) * 2.0)
    pr_hetero.append(f_hetero / len(first_SNP) * 2.0)
    pr_min.append(f_min / len(first_SNP) * 2.0)
    

In [ ]:
#calculating prob inferred
pr_maj_inf = []
pr_hetero_inf = []
pr_min_inf = []

for loc in range(5000):
    f_maj = 0
    f_hetero = 0
    f_min = 0
    first_SNP = inf_file.loc[loc, 5:]

    for i in range(int((len(first_SNP))/3)):
        f_maj += first_SNP.loc[5 +3*i]
        f_hetero += first_SNP.loc[5 + 3*i + 1]
        f_min += first_SNP.loc[5 +3*i + 2]
    pr_maj_inf.append(f_maj / len(first_SNP) * 2.0)
    pr_hetero_inf.append(f_hetero / len(first_SNP) * 2.0)
    pr_min_inf.append(f_min / len(first_SNP) * 2.0)
    

In [ ]:
#calculating Hellinger score

import math
H = []
for i in range(len(pr_maj)):
    sqrt_sum = math.sqrt(pr_maj[i] * pr_maj_inf[i]) + math.sqrt(pr_hetero[i] * pr_hetero_inf[i]) + \
                        math.sqrt(pr_min[i] * pr_min_inf[i])
    #sqrt_sum = math.sqrt(pr_maj[i] * pr_maj[i]) + math.sqrt(pr_hetero[i] * pr_hetero[i]) + \
    #                    math.sqrt(pr_min[i] * pr_min[i])
    H.append(1 - math.sqrt(1-sqrt_sum))

In [ ]:
print(H[:10])
print(max(H))
print(len(H))
print(min(H))

In [ ]:
#want to calculate indices where imputation actually happened (rather than the SNPs \
#that were preserved)

imputed_indices = []
preserved_indices = []

for loc in range(len(pr_maj)):
    chrom = inf_file.loc[loc, 0]
    if chrom == "22":
        preserved_indices.append(loc)
    else:
        imputed_indices.append(loc)
        


In [ ]:
#calculating Hellinger score

import math
H_only_imputed = []
for i in range(len(imputed_indices)):
    sqrt_sum = math.sqrt(pr_maj[imputed_indices[i]] * pr_maj_inf[imputed_indices[i]]) + \
        math.sqrt(pr_hetero[imputed_indices[i]] * pr_hetero_inf[imputed_indices[i]]) + \
        math.sqrt(pr_min[imputed_indices[i]] * pr_min_inf[imputed_indices[i]])
    #sqrt_sum = math.sqrt(pr_maj[i] * pr_maj[i]) + math.sqrt(pr_hetero[i] * pr_hetero[i]) + \
    #                    math.sqrt(pr_min[i] * pr_min[i])
    H_only_imputed.append(1 - math.sqrt(1-sqrt_sum))

In [ ]:
print(max(H_only_imputed))

In [ ]:
print(H_only_imputed[:10])

In [ ]:
print(H[:10])

In [ ]:
#making violin plot of Hellinger score
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.patches as mpatches
from itertools import repeat

red_patch = mpatches.Patch(color='blue')


fig = plt.figure()

ax = fig.add_axes([0,0,1,1])
#ax.set_ylim(0, 1)

#fake data
pos   = [1]
label = ['IMPUTE2 at 50%']
data  = [np.random.normal(size=100) for i in pos]
fake_handles = repeat(red_patch, len(pos))


bp = ax.violinplot(H_only_imputed)
ax.legend(fake_handles, label)
plt.show()